In [1]:
import numpy as np
import os
import sys
import torch
import pytorch3d

import matplotlib.pyplot as plt

import trimesh
from pytorch3d.io import load_objs_as_meshes, save_obj
import numpy as np
from tqdm.notebook import tqdm



from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
from pytorch3d.io import load_objs_as_meshes, save_obj

from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    OpenGLPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    SoftSilhouetteShader,
    SoftPhongShader,
    TexturesVertex,
)

from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)

from src.plot_image_grid import image_grid
from src.cleansed_cube import Ellipsoid

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")


obj_filename = "./data/bunny.obj"
obj_filename = "./data/nefertiti.obj"
bunny = load_objs_as_meshes([obj_filename], device=device)

textures=TexturesVertex(verts_features=[torch.ones_like(bunny.verts_packed())])

bunny = Meshes(verts=[bunny.verts_packed()], 
           faces=[bunny.faces_packed()],
           textures=textures)

verts = bunny.verts_packed()
N = verts.shape[0]
center = verts.mean(0)
scale = max((verts - center).abs().max(0)[0])
bunny.offset_verts_(-center)
bunny.scale_verts_((1.0 / float(scale)));

In [7]:
n = 64

ellipsoid = Ellipsoid(n).to(device)
rs = torch.randn(3, requires_grad=True, device=device)

#optimizer = torch.optim.SGD([rs], lr=1., momentum=0.)
optimizer = torch.optim.Adam([rs], lr=0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [8]:
Niter = 20001
num_views_per_iteration = 2

loop = tqdm(range(Niter))

laplace_weight =  1.

w_chamfer = 1.0
w_laplacian = 0.1 
w_normal = 0.01 
w_edge = 1.0 

for i in loop:
    stop = False
    
    optimizer.zero_grad()
    
    new_src_mesh = ellipsoid(torch.sigmoid(rs))
    verts, faces, colors = ellipsoid(torch.sigmoid(rs))

    textures = TexturesVertex(verts_features=[torch.ones_like(verts)])
    p3d_mesh = Meshes(verts=[verts],
                  faces=[faces],
                  textures=textures)
    
    sample_src = sample_points_from_meshes(p3d_mesh, 5000)
    sample_trg = sample_points_from_meshes(bunny, 5000)
    
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)
    loss_laplacian = mesh_laplacian_smoothing(p3d_mesh, method="uniform")
    loss_normal = mesh_normal_consistency(p3d_mesh)
    loss_edge = mesh_edge_loss(p3d_mesh)
    
    
    
    
    #laplacian_smoothing = mesh_laplacian_smoothing(new_src_mesh, method="uniform")
    sum_loss = torch.tensor(0.0, device=device) 
    
    sum_loss += loss_chamfer * w_chamfer
    #sum_loss += laplace_loss * w_laplacian
    sum_loss += loss_laplacian * w_laplacian
    sum_loss += loss_normal * w_normal
    sum_loss += loss_edge * w_edge
    
    loop.set_description("total_loss = %.6f" % sum_loss)
    
    sum_loss.backward()
    #cube.smooth()
    optimizer.step()
    
    if i % 500 == 0:
        f = f'./data/ellipsoid_mesh_{n}_{i}.obj'
        ellipsoid.export(f, torch.sigmoid(rs))
        print(f)
#kernel = 0
f = f'./data/ellipsoid_mesh_{n}_{kernel}.obj'
ellipsoid.export(f, torch.sigmoid(rs))
f

  0%|          | 0/20001 [00:00<?, ?it/s]

./data/ellipsoid_mesh_64_0.obj
./data/ellipsoid_mesh_64_500.obj
./data/ellipsoid_mesh_64_1000.obj
./data/ellipsoid_mesh_64_1500.obj
./data/ellipsoid_mesh_64_2000.obj
./data/ellipsoid_mesh_64_2500.obj
./data/ellipsoid_mesh_64_3000.obj
./data/ellipsoid_mesh_64_3500.obj
./data/ellipsoid_mesh_64_4000.obj
./data/ellipsoid_mesh_64_4500.obj
./data/ellipsoid_mesh_64_5000.obj
./data/ellipsoid_mesh_64_5500.obj
./data/ellipsoid_mesh_64_6000.obj
./data/ellipsoid_mesh_64_6500.obj
./data/ellipsoid_mesh_64_7000.obj
./data/ellipsoid_mesh_64_7500.obj
./data/ellipsoid_mesh_64_8000.obj
./data/ellipsoid_mesh_64_8500.obj
./data/ellipsoid_mesh_64_9000.obj
./data/ellipsoid_mesh_64_9500.obj
./data/ellipsoid_mesh_64_10000.obj
./data/ellipsoid_mesh_64_10500.obj
./data/ellipsoid_mesh_64_11000.obj
./data/ellipsoid_mesh_64_11500.obj
./data/ellipsoid_mesh_64_12000.obj
./data/ellipsoid_mesh_64_12500.obj
./data/ellipsoid_mesh_64_13000.obj
./data/ellipsoid_mesh_64_13500.obj
./data/ellipsoid_mesh_64_14000.obj
./data/el

NameError: name 'kernel' is not defined